In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from sklearn.linear_model import Ridge, Lasso
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score
from sklearn.preprocessing import MinMaxScaler
from skopt import gp_minimize
from skopt.space import Real

# Output directory
output_dir = "Excel_Ridge_Lasso_Bayes_Results"
os.makedirs(output_dir, exist_ok=True)

# Load and normalize features
FEATURE_COLUMNS = [
    "C", "H", "O", "N", "F", "S", "System_Size", "a", "b", "c",
    "alpha", "beta", "gamma", "density", "PLD", "LCD", "N2_SA",
    "Probe_Accessible", "Probe_Occupiable", "Rosenbluth_Weight"
]
data = pd.read_csv("PIM_ExpFeatures.csv")
features = data[FEATURE_COLUMNS].values
labels = pd.read_csv("PIM_Qst_Labels.csv")["Qst_CO2_298K"].values

scaler = MinMaxScaler()
features = scaler.fit_transform(features)

# Define common optimization space
space = [Real(1e-6, 10.0, prior="log-uniform", name="alpha")]

def optimize_model(model_class, model_name):
    def objective(params):
        alpha = params[0]
        r2_scores = []

        kf = KFold(n_splits=5, shuffle=True, random_state=42)
        for train_idx, test_idx in kf.split(features):
            X_train, X_test = features[train_idx], features[test_idx]
            y_train, y_test = labels[train_idx], labels[test_idx]

            model = model_class(alpha=alpha, max_iter=10000)
            model.fit(X_train, y_train)
            preds = model.predict(X_test)
            r2_scores.append(r2_score(y_test, preds))

        return -np.mean(r2_scores)

    # Run Bayesian optimization
    result = gp_minimize(
        objective,
        space,
        n_calls=500,
        n_initial_points=50,
        random_state=42,
        verbose=True
    )
    return result

# Optimize both Ridge and Lasso
results = {}
for model_class, name in [(Ridge, "Ridge"), (Lasso, "Lasso")]:
    print(f"\n🔍 Optimizing {name}...")
    result = optimize_model(model_class, name)
    best_alpha = result.x[0]
    results[name] = best_alpha
    #pd.DataFrame({"Best Alpha": [best_alpha]}).to_csv(
    #    os.path.join(output_dir, f"{name}_best_alpha.csv"), index=False
    #)

    # Final evaluation using best alpha
    model = model_class(alpha=best_alpha, max_iter=10000)
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    train_r2s, test_r2s = [], []
    all_train_actuals, all_train_preds = [], []
    all_test_actuals, all_test_preds = [], []

    for train_idx, test_idx in kf.split(features):
        X_train, X_test = features[train_idx], features[test_idx]
        y_train, y_test = labels[train_idx], labels[test_idx]

        model.fit(X_train, y_train)
        train_preds = model.predict(X_train)
        test_preds = model.predict(X_test)

        all_train_actuals.extend(y_train)
        all_train_preds.extend(train_preds)
        all_test_actuals.extend(y_test)
        all_test_preds.extend(test_preds)

        train_r2s.append(r2_score(y_train, train_preds))
        test_r2s.append(r2_score(y_test, test_preds))
        
        # Final parity plot with both training and testing results
        plt.figure(figsize=(6, 6))
        plt.rcParams.update({'font.family': 'Times New Roman'})

        # Scatter points
        plt.scatter(all_train_actuals, all_train_preds, color='blue', alpha=0.7, edgecolor='k', label=f"Training ($R^2$ = {np.mean(train_r2s):.3f})")
        plt.scatter(all_test_actuals, all_test_preds, color='red', alpha=0.7, edgecolor='k', label=f"Testing ($R^2$ = {np.mean(test_r2s):.3f})")

        # Diagonal reference line
        min_val = min(min(all_train_actuals), min(all_test_actuals))
        max_val = max(max(all_train_actuals), max(all_test_actuals))
        plt.plot([min_val, max_val], [min_val, max_val], 'k--', linewidth=1)

        # Labels and title
        plt.xlabel("Actual Qst", fontsize=14)
        plt.ylabel("Predicted Qst", fontsize=14)
        plt.title(f"{name} Prediction Results", fontsize=14)

        # Styling
        plt.xticks(fontsize=12, color='black')
        plt.yticks(fontsize=12, color='black')
        plt.grid(True, linestyle='--', linewidth=0.5)
        plt.legend(loc='upper left', fontsize=12, frameon=False)
        plt.gca().set_aspect('equal', adjustable='box')
        plt.tight_layout()

        # Save as PDF
        plt.savefig(os.path.join(output_dir, f"{name}_combined_parity_plot.pdf"), format='pdf')
        plt.close()

    # Save R² results
#    cv_df = pd.DataFrame({
#        "Fold": range(1, 6),
#        "Training R^2": train_r2s,
#        "Testing R^2": test_r2s
#    })
#    cv_df.to_csv(os.path.join(output_dir, f"{name}_cv_results.csv"), index=False)

    # Parity plots
#    for label, actual, pred, r2, fname in zip(
#        ["Training", "Testing"],
#        [all_train_actuals, all_test_actuals],
#        [all_train_preds, all_test_preds],
#        [np.mean(train_r2s), np.mean(test_r2s)],
#        [f"{name}_training_parity_plot.png", f"{name}_testing_parity_plot.png"]
#    ):
#        plt.figure()
#        plt.scatter(actual, pred, alpha=0.7, label=f"{label} Data")
#        plt.plot([min(actual), max(actual)], [min(actual), max(actual)], 'k--', label="Perfect Prediction")
#        plt.xlabel("Actual")
#        plt.ylabel("Predicted")
#        plt.title(f"{name} {label} Parity Plot ($R^2 = {r2:.3f}$)")
#        plt.legend()
#        plt.grid()
#        plt.savefig(os.path.join(output_dir, fname))
#        plt.close()

#print("✅ Ridge and Lasso with Bayesian optimization complete.")



🔍 Optimizing Ridge...
Iteration No: 1 started. Evaluating function at random point.
Iteration No: 1 ended. Evaluation done at random point.
Time taken: 0.0021
Function value obtained: -0.8970
Current minimum: -0.8970
Iteration No: 2 started. Evaluating function at random point.
Iteration No: 2 ended. Evaluation done at random point.
Time taken: 0.0017
Function value obtained: -0.9547
Current minimum: -0.9547
Iteration No: 3 started. Evaluating function at random point.
Iteration No: 3 ended. Evaluation done at random point.
Time taken: 0.0016
Function value obtained: -0.9085
Current minimum: -0.9547
Iteration No: 4 started. Evaluating function at random point.
Iteration No: 4 ended. Evaluation done at random point.
Time taken: 0.0016
Function value obtained: -0.9521
Current minimum: -0.9547
Iteration No: 5 started. Evaluating function at random point.
Iteration No: 5 ended. Evaluation done at random point.
Time taken: 0.0016
Function value obtained: -0.9544
Current minimum: -0.9547
It

/Users/samusmaster210/.conda/envs/Chem5/lib/python3.9/site-packages/skopt/optimizer/optimizer.py:517: UserWarning: The objective has been evaluated at point [1.2314965050634562e-06] before, using random point [6.54406969080621e-06]
  warnings.warn(


Iteration No: 127 ended. Search finished for the next optimal point.
Time taken: 1.1904
Function value obtained: -0.9547
Current minimum: -0.9547
Iteration No: 128 started. Searching for the next optimal point.
Iteration No: 128 ended. Search finished for the next optimal point.
Time taken: 1.1799
Function value obtained: -0.9546
Current minimum: -0.9547
Iteration No: 129 started. Searching for the next optimal point.
Iteration No: 129 ended. Search finished for the next optimal point.
Time taken: 1.2927
Function value obtained: -0.9547
Current minimum: -0.9547
Iteration No: 130 started. Searching for the next optimal point.
Iteration No: 130 ended. Search finished for the next optimal point.
Time taken: 1.2445
Function value obtained: -0.9533
Current minimum: -0.9547
Iteration No: 131 started. Searching for the next optimal point.
Iteration No: 131 ended. Search finished for the next optimal point.
Time taken: 1.3282
Function value obtained: -0.9547
Current minimum: -0.9547
Iteration 

/Users/samusmaster210/.conda/envs/Chem5/lib/python3.9/site-packages/skopt/optimizer/optimizer.py:517: UserWarning: The objective has been evaluated at point [1.2130277338253326e-06] before, using random point [3.072914417726414]
  warnings.warn(


Iteration No: 253 ended. Search finished for the next optimal point.
Time taken: 4.3479
Function value obtained: -0.7464
Current minimum: -0.9547
Iteration No: 254 started. Searching for the next optimal point.
Iteration No: 254 ended. Search finished for the next optimal point.
Time taken: 4.2546
Function value obtained: -0.9547
Current minimum: -0.9547
Iteration No: 255 started. Searching for the next optimal point.
Iteration No: 255 ended. Search finished for the next optimal point.
Time taken: 4.3833
Function value obtained: -0.9547
Current minimum: -0.9547
Iteration No: 256 started. Searching for the next optimal point.
Iteration No: 256 ended. Search finished for the next optimal point.
Time taken: 4.0853
Function value obtained: -0.9547
Current minimum: -0.9547
Iteration No: 257 started. Searching for the next optimal point.
Iteration No: 257 ended. Search finished for the next optimal point.
Time taken: 4.5435
Function value obtained: -0.9545
Current minimum: -0.9547
Iteration 

...

Iteration No: 309 ended. Search finished for the next optimal point.
Time taken: 6.4112
Function value obtained: -0.9547
Current minimum: -0.9575
Iteration No: 310 started. Searching for the next optimal point.
Iteration No: 310 ended. Search finished for the next optimal point.
Time taken: 6.8342
Function value obtained: -0.9574
Current minimum: -0.9575
Iteration No: 311 started. Searching for the next optimal point.
Iteration No: 311 ended. Search finished for the next optimal point.
Time taken: 6.3878
Function value obtained: -0.9574
Current minimum: -0.9575
Iteration No: 312 started. Searching for the next optimal point.
Iteration No: 312 ended. Search finished for the next optimal point.
Time taken: 6.2137
Function value obtained: -0.9574
Current minimum: -0.9575
Iteration No: 313 started. Searching for the next optimal point.
Iteration No: 313 ended. Search finished for the next optimal point.
Time taken: 7.0445
Function value obtained: -0.9573
Current minimum: -0.9575
Iteration 

/Users/samusmaster210/.conda/envs/Chem5/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.062e-01, tolerance: 1.803e-02
  model = cd_fast.enet_coordinate_descent(
/Users/samusmaster210/.conda/envs/Chem5/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.711e-01, tolerance: 1.852e-02
  model = cd_fast.enet_coordinate_descent(
/Users/samusmaster210/.conda/envs/Chem5/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the featu

Iteration No: 434 ended. Search finished for the next optimal point.
Time taken: 11.3243
Function value obtained: -0.9547
Current minimum: -0.9575
Iteration No: 435 started. Searching for the next optimal point.
Iteration No: 435 ended. Search finished for the next optimal point.
Time taken: 11.7014
Function value obtained: -0.9574
Current minimum: -0.9575
Iteration No: 436 started. Searching for the next optimal point.
Iteration No: 436 ended. Search finished for the next optimal point.
Time taken: 11.1984
Function value obtained: -0.9574
Current minimum: -0.9575
Iteration No: 437 started. Searching for the next optimal point.
Iteration No: 437 ended. Search finished for the next optimal point.
Time taken: 12.1121
Function value obtained: -0.9574
Current minimum: -0.9575
Iteration No: 438 started. Searching for the next optimal point.
Iteration No: 438 ended. Search finished for the next optimal point.
Time taken: 11.8392
Function value obtained: -0.9574
Current minimum: -0.9575
Itera

/Users/samusmaster210/.conda/envs/Chem5/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.671e-01, tolerance: 1.803e-02
  model = cd_fast.enet_coordinate_descent(
/Users/samusmaster210/.conda/envs/Chem5/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.740e-01, tolerance: 1.852e-02
  model = cd_fast.enet_coordinate_descent(
/Users/samusmaster210/.conda/envs/Chem5/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the featu

Iteration No: 487 ended. Search finished for the next optimal point.
Time taken: 14.1599
Function value obtained: -0.9547
Current minimum: -0.9575
Iteration No: 488 started. Searching for the next optimal point.
Iteration No: 488 ended. Search finished for the next optimal point.
Time taken: 13.2047
Function value obtained: -0.9574
Current minimum: -0.9575
Iteration No: 489 started. Searching for the next optimal point.
Iteration No: 489 ended. Search finished for the next optimal point.
Time taken: 13.9408
Function value obtained: -0.9574
Current minimum: -0.9575
Iteration No: 490 started. Searching for the next optimal point.
Iteration No: 490 ended. Search finished for the next optimal point.
Time taken: 14.4473
Function value obtained: -0.9574
Current minimum: -0.9575
Iteration No: 491 started. Searching for the next optimal point.
Iteration No: 491 ended. Search finished for the next optimal point.
Time taken: 14.5450
Function value obtained: -0.9574
Current minimum: -0.9575
Itera

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from sklearn.linear_model import Ridge, Lasso
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score
from sklearn.preprocessing import MinMaxScaler
import shap

# Output directory
output_dir = "Excel_Ridge_Lasso_Bayes_Results"
os.makedirs(output_dir, exist_ok=True)

# Load and normalize features
FEATURE_COLUMNS = [
    "C", "H", "O", "N", "F", "S", "System_Size", "a", "b", "c",
    "alpha", "beta", "gamma", "density", "PLD", "LCD", "N2_SA",
    "Probe_Accessible", "Probe_Occupiable", "Rosenbluth_Weight"
]
data = pd.read_csv("PIM_ExpFeatures.csv")
features = data[FEATURE_COLUMNS].values
labels = pd.read_csv("PIM_Qst_Labels.csv")["Qst_CO2_298K"].values

scaler = MinMaxScaler()
features = scaler.fit_transform(features)

# Retrain best CatBoost model on full dataset
final_model = Lasso(alpha=0.000678645, max_iter=10000)
final_model.fit(features, labels)

# SHAP analysis
explainer = shap.LinearExplainer(final_model, features)
shap_values = explainer(features)

# Save SHAP values (one row per sample, one column per feature)
shap_df = pd.DataFrame(shap_values.values, columns=FEATURE_COLUMNS)
shap_df["Sample_Index"] = np.arange(len(shap_df))
shap_df.to_csv(os.path.join(output_dir, "Lasso_SHAP_values.csv"), index=False)

# Compute and save feature ranking (mean absolute SHAP)
mean_abs_shap = np.abs(shap_df[FEATURE_COLUMNS]).mean().sort_values(ascending=False)
mean_abs_shap_df = mean_abs_shap.reset_index()
mean_abs_shap_df.columns = ["Feature", "Mean_Absolute_SHAP"]
mean_abs_shap_df.to_csv(os.path.join(output_dir, "Lasso_SHAP_feature_ranking.csv"), index=False)

print("✅ SHAP analysis completed and saved.")

✅ SHAP analysis completed and saved.


In [6]:
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib as mpl

# Set global font to Times New Roman and base font size to 14
mpl.rcParams['font.family'] = 'Times New Roman'
mpl.rcParams['font.size'] = 14

# Load SHAP feature ranking
shap_ranking_path = os.path.join(output_dir, "Lasso_SHAP_feature_ranking.csv")
shap_ranking = pd.read_csv(shap_ranking_path)

# Plot: Top 10 SHAP features
plt.figure(figsize=(8, 6))
barplot = sns.barplot(
    data=shap_ranking.head(10),
    x="Mean_Absolute_SHAP",
    y="Feature",
    orient="h",
    palette="viridis"
)

# Adjust axis labels and tick label sizes
barplot.set_title("Top 10 Feature Importances by SHAP (Lasso)", fontsize=14)
barplot.set_xlabel("Mean Absolute SHAP Value", fontsize=14)
barplot.set_ylabel("Feature", fontsize=14)
barplot.tick_params(axis='x', labelsize=12)
barplot.tick_params(axis='y', labelsize=14)
plt.grid(True, axis='x', linestyle='--', alpha=0.7)
plt.tight_layout()

# Save the figure
plot_path = os.path.join(output_dir, "Lasso_SHAP_feature_importance_plot.png")
plt.savefig(plot_path, dpi=300)
plt.close()

print(f"✅ SHAP feature importance plot saved to: {plot_path}")


/var/folders/3q/n2zkzqzd5q9c607jywjkk4tc0000gn/T/ipykernel_77396/3226364334.py:15: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.

  barplot = sns.barplot(


✅ SHAP feature importance plot saved to: Excel_Ridge_Lasso_Bayes_Results/Lasso_SHAP_feature_importance_plot.png
